# An AMPL model for the transportation problem

In [1]:
# install dependencies
%pip install -q amplpy pandas -U

from amplpy import AMPL, ampl_notebook
import pandas as pd

ampl = ampl_notebook(
    modules=['highs', 'cbc'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics

Note: you may need to restart the kernel to use updated packages.


Two fundamental sets of objects underlie the
transportation problem:  the sources or origins (mills, in our example) and the
destinations (factories).  Thus we begin the `AMPL` model with a
declaration of these two sets:
```
set ORIG;
set DEST;
```
There is a supply of something at each origin (tons of steel coils produced,
in our case), and a demand for the same thing at each destination
(tons of coils ordered).
`AMPL` defines
nonnegative quantities like these with
`param`
statements indexed over a set; in this case we add
one extra refinement, a
`check`
statement to test the data for validity:
```
param supply {ORIG} >= 0;
param demand {DEST} >= 0;

check: sum {i in ORIG} supply[i] = sum {j in DEST} demand[j];
```
The
`check`
statement says that the sum of the supplies has to equal
the sum of the demands.  The way that our model is to be set up, there
can't possibly be any solutions unless this condition is satisfied.  By
putting it in a
`check`
statement, we tell `AMPL` to test this condition
after reading the data, and to issue an error
message if it is violated.

For each combination of an origin and a destination, there is a
transportation cost and a variable representing the amount
transported.  Again, the ideas from previous chapters are easily adapted to
produce the appropriate `AMPL` statements:
```
param cost {ORIG,DEST} >= 0;
var Trans {ORIG,DEST} >= 0;
```
For a particular origin
`i`
and destination
`j` ,
we ship
`Trans[i,j]`
units
from
`i`
to
`j` ,
at a cost of
`cost[i,j]`
per unit; the total cost for
this pair is
```
cost[i,j] * Trans[i,j]
```
Adding over all pairs, we have
the objective function:
```
minimize Total_Cost:
   sum {i in ORIG, j in DEST} cost[i,j] * Trans[i,j];
```
which could also be written as
```
   sum {j in DEST, i in ORIG} cost[i,j] * Trans[i,j];
```
or as
```
   sum {i in ORIG} sum {j in DEST} cost[i,j] * Trans[i,j];
```
As long as you
express the objective in some mathematically correct way, `AMPL` will
sort out the terms.

It remains to specify the two collections of constraints, those at
the origins and those at the destinations.  If we name these collections
`Supply`
and
`Demand` ,
their declarations will start as follows:
```
subject to Supply {i in ORIG}:  ...
subject to Demand {j in DEST}:  ...
```
To complete the
`Supply`
constraint for origin
`i` ,
we need to say that the sum
of all shipments out of
`i`
is equal to the supply available.  Since the amount
shipped out of
`i`
to a particular destination
`j`
is
`Trans[i,j]` ,
the amount
shipped to all destinations must be
```
sum {j in DEST} Trans[i,j]
```
Since
we have already defined a parameter
`supply`
indexed over origins, the
amount available at
`i`
is
`supply[i]` .
Thus the constraint is
```
subject to Supply {i in ORIG}:
  sum {j in DEST} Trans[i,j] = supply[i];
```
(Note that the names
`supply`
and
`Supply`
are unrelated;
`AMPL` distinguishes upper and lower case.)  The other collection of
constraints is much the same, except that the roles of
`i`
in
`ORIG` ,
and
`j`
in
`DEST` ,
are exchanged, and the sum equals
`demand[j]` .

We can now present the complete transportation model,
Figure @Tut3@-@transp.mod@.



In [2]:
%%writefile transp.mod

set ORIG;   # origins
set DEST;   # destinations

param supply {ORIG} >= 0;   # amounts available at origins
param demand {DEST} >= 0;   # amounts required at destinations

   check: sum {i in ORIG} supply[i] = sum {j in DEST} demand[j];

param cost {ORIG,DEST} >= 0;   # shipment costs per unit
var Trans {ORIG,DEST} >= 0;    # units to be shipped

minimize Total_Cost:
   sum {i in ORIG, j in DEST} cost[i,j] * Trans[i,j];

subject to Supply {i in ORIG}:
   sum {j in DEST} Trans[i,j] = supply[i];

subject to Demand {j in DEST}:
   sum {i in ORIG} Trans[i,j] = demand[j];

Overwriting transp.mod


.F1
``` .2i
.get transp.mod
```
.C "Figure @Tut3@-@transp.mod@" "Transportation model (\f(CWtransp.mod\fP)."
.ix file~[transp.mod]
.F2

As you might have noticed, we have been consistent in using the index
`i`
to run over the set
`ORIG` ,
and
the index
`j`
to run over
`DEST` .
This is not an `AMPL` requirement,
but such a convention makes it easier to read a model.
You may name your own indices
whatever you like, but keep in mind that
the scope of an index 
&ndash; 
the part of the model
where it has the same meaning 
&ndash;
is to the end of the expression
that defines it.
Thus in the
`Demand`
constraint
```
subject to Demand {j in DEST}:
  sum {i in ORIG} Trans[i,j] = demand[j];
```
the scope of
`j`
runs to the semicolon at the end of the declaration, while the scope of
`i`
extends only through the summand
`Trans[i,j]` .
Since
`i` 's
scope is inside
`j` 's
scope, these two indices must have different
names.  Also an index may not have the same name as a set or other model
component.  Index scopes are discussed more fully, with further examples,
in Section @Sets1@.@indexing-exprs@.

Data values for the transportation model are shown in Figure @Tut3@-@transp.dat@.
To define
`DEST`
and
`demand` ,
we have used an input format that
permits a set and one or more parameters indexed over it to be specified
together.
The set name is surrounded by colons.
(We also show some comments, which can appear among data statements
just as in a model.)

.F1
``` .1i
.get transp.dat
```


In [3]:
df_orig = pd.DataFrame(
    [
        ['GARY', 1400],
        ['CLEV', 2600],
        ['PITT', 2900]
    ],
    columns=['ORIG', 'supply']
).set_index('ORIG')

df_dest = pd.DataFrame(
    [
        ['FRA', 900],
        ['DET', 1200],
        ['LAN', 600],
        ['WIN', 400],
        ['STL', 1700],
        ['FRE', 1100],
        ['LAF', 1000]
    ],
    columns=['DEST', 'demand']
).set_index('DEST')

df_cost = pd.DataFrame(
    [
        ['GARY', 39, 14, 11, 14, 16, 82, 8],
        ['CLEV', 27,  9, 12, 9, 26, 95, 17],
        ['PITT', 24, 14, 17, 13, 28, 99, 20]
    ],
    columns=['ORIG', 'FRA', 'DET', 'LAN', 'WIN', 'STL', 'FRE', 'LAF']
).set_index('ORIG')

display(df_orig)
display(df_dest)
display(df_cost)

,supply
ORIG,
GARY,1400
CLEV,2600
PITT,2900


,demand
DEST,
FRA,900
DET,1200
LAN,600
WIN,400
STL,1700
FRE,1100
LAF,1000


,FRA,DET,LAN,WIN,STL,FRE,LAF
ORIG,,,,,,,
GARY,39,14,11,14,16,82,8
CLEV,27,9,12,9,26,95,17
PITT,24,14,17,13,28,99,20


If the model is stored in a file
transp.mod
and the
data in
transp.dat ,
we can solve the linear program and
examine the output:

In [7]:
ampl = AMPL()
ampl.read('transp.mod')
ampl.set_data(df_orig, 'ORIG')
ampl.set_data(df_dest, 'DEST')
ampl.param['cost'] = df_cost
ampl.option['solver'] = 'highs'
ampl.solve()
ampl.display('Trans')

HiGHS 1.6.0: HiGHS 1.6.0: optimal solution; objective 196200
10 simplex iterations
0 barrier iterations
 
Trans [*,*] (tr)
:     CLEV   GARY   PITT    :=
DET   1200      0      0
FRA      0      0    900
FRE      0   1100      0
LAF    400    300    300
LAN    600      0      0
STL      0      0   1700
WIN    400      0      0
;



By displaying the variable
`Trans` ,
we see that most destinations are supplied
from a single mill, but
\s-2CLEV\s0, \s-2GARY\s0 and \s-2PITT\s0 all ship to \s-2LAF\s0.

It is instructive to compare this solution to one given by another solver,

In [10]:
ampl.option['solver'] = 'snopt'
ampl.solve()
ampl.display('Trans')

SNOPT 7.5-1.2 : Optimal solution found.
0 iterations, objective 196200
Trans [*,*] (tr)
:     CLEV     GARY        PITT      :=
DET   1200      0           0
FRA      0      0         900
FRE      0   1100           0
LAF    400    237.723     362.277
LAN    600      0           0
STL      0     62.2768   1637.72
WIN    400      0           0
;



The minimum cost is still 196200, but it is achieved in
a different way.  Alternative optimal solutions such as these are often
exhibited by transportation problems, particularly when the coefficients in
the objective function are round numbers.

Unfortunately, there is no easy way to characterize all the
optimal solutions.  You may be able to get a better choice of optimal
solution by working with several objectives, however, as we will illustrate in
Section @LPs@.@lp-objectives@.